# Modelos probabilísticos

Los ejemplos y la discusión que sigue está parcialmente tomado del libro:

[*Introduction to Machine Learning with Python*](http://shop.oreilly.com/product/0636920030515.do)  
**Andreas C. Müller & Sarah Guido**  
O'Reilly 2017

En concreto, este tema está basado en el capítulo 7, pero con modificaciones. 

Github con el material del libro: [Github](https://github.com/amueller/introduction_to_ml_with_python). 

El libro está accesible *online* desde la [Biblioteca de la Universidad de Sevilla](https://fama.us.es), como recurso electrónico.

## Aplicación: clasificación de textos usando Naive Bayes

### Cargando el conjunto de entrenamiento: críticas de película en IMDB 

Como datos en nuestra aplicación usaremos críiticas de películas en la web IMDB (Internet Movie Database). Son críticas que ya vienen con la etiqueta "pos" o "neg", de acuerdo a la puntuación que acompaña a la crítica (positiva, 7 o más; negativa 4 o menos). El objetivo es ser capaz de declarar como positiva o negativa una crítica (por supuesto, sin saber la puntuación que la acompaña).

Los datos están disponibles en [http://ai.stanford.edu/~amaas/data/sentiment/](http://ai.stanford.edu/~amaas/data/sentiment/)

Descargar el archivo comprimido con los datos, y descomprimir, quedando la siguiente estructura: 

```
data/
└── aclImdb/
    ├── test/
    │   ├── neg
    │   └── pos
    └── train/
        ├── neg
        └── pos
```
     
Nota: de aquí hemos eliminado la carpete `train/unsup`, que no usaremos.

La función `load_files` que viene en el módulo `datasets` de scikit-learn permite cargar conjuntos de datos que vienen en carpetas con esa estructura. Cargamos el conjunto de entrenamiento:

In [1]:
from sklearn.datasets import load_files

In [2]:
reviews_train = load_files("data/aclImdb/train/")

In [3]:
text_train, y_train = reviews_train.data, reviews_train.target

In [4]:
print("Tipo de text_train: {}".format(type(text_train)))
print("Tipo de y_train: {}".format(type(y_train)))
print("Cantidad de textos en el conjunto de entrenamiento: {}".format(len(text_train)))
print("Etiquetas: {}".format(reviews_train.target_names))

Tipo de text_train: <class 'list'>
Tipo de y_train: <class 'numpy.ndarray'>
Cantidad de textos en el conjunto de entrenamiento: 25000
Etiquetas: ['neg', 'pos']


Dos ejemplo de estas revisiones que acabamos de cargar. Téngase en cuenta que los valores de clasificación son 0 y 1, correspondiendo a "neg" y "pos", resp..

In [5]:
print("text_train[6]:\n{}\n".format(text_train[6]))
print("y_train[6]: {}\n".format(y_train[6]))
print("Etiqueta asociada: {}".format(reviews_train.target_names[y_train[6]]))

text_train[6]:
b"This movie has a special way of telling the story, at first i found it rather odd as it jumped through time and I had no idea whats happening.<br /><br />Anyway the story line was although simple, but still very real and touching. You met someone the first time, you fell in love completely, but broke up at last and promoted a deadly agony. Who hasn't go through this? but we will never forget this kind of pain in our life. <br /><br />I would say i am rather touched as two actor has shown great performance in showing the love between the characters. I just wish that the story could be a happy ending."

y_train[6]: 1

Etiqueta asociada: pos


In [6]:
print("text_train[25]:\n{}\n".format(text_train[25]))
print("y_train[25]: {}\n".format(y_train[25]))
print("Etiqueta asociada: {}".format(reviews_train.target_names[y_train[25]]))

text_train[25]:
b"A chemical spill is turning people into zombies. It's up to two doctor's to survive the epidemic. It's an Andreas Schnaas film so you know what the par for the course will be. Bad acting, horribly awful special effects, and no budget to speak of. The dubbing is ridiculous with a capital R and the saddest thing is that I feel compelled to write one word about this piece of excrement, much less the ten lines mandatory because of the guidelines placed on me by IMDb. My original review of merely one word: Crap wouldn't fly so I have to revise it and go more in to how bad it is. But I don't know if I can, so.. wait I think I may have enough words, or lines rather to make this review pass. Which is cool, I guess. So in summation: This movie sucks balls, don't watch it.<br /><br />My Grade: F"

y_train[25]: 0

Etiqueta asociada: neg


Como se observa en los ejemplos anteriores, en los textos hay muchas etiquetas de cambio de línea en HTML, que obviamente no tienen impacto en el sentimiento que expresa el texto, así que las quitamos:

In [7]:
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]

Se trata además de un conjunto de datos balanceado:

In [8]:
import numpy as np
print("Ejemplos por cada clase: {}".format(np.bincount(y_train)))

Ejemplos por cada clase: [12500 12500]


Cargamos de la msma manera el conjunto de test, que tiene exactamente el mismo número de ejemplos, e igualmente balanceados:

In [9]:
reviews_test = load_files("data/aclImdb/test/")

In [10]:
text_test, y_test = reviews_test.data, reviews_test.target

In [11]:
print("Número de ejemplos en test: {}".format(len(text_test)))
print("Ejemplos por cada clase: {}".format(np.bincount(y_test)))
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

Número de ejemplos en test: 25000
Ejemplos por cada clase: [12500 12500]


### El modelo vectorial *Bag of Words*

Antes de poder aplicar modelos de aprendizaje a textos, debemos representar los documentos mediante vectores numéricos. Como hemos visto en el desarrollo del tema, la forma más fácil de hacerlo es, una vez fijado los términos de nuestro *Vocabulario* (y un orden implícito entre los términos), mediante un vector en el que en cada componente tenemos el número de veces que aparece el correspondiente término del vocabulario, en el documento.

#### Un ejemplo de juguete sobre la vectorización:

In [12]:
bards_words =["The fool doth think he is wise,",
              "but the wise man knows himself to be a fool"]

El transformador `CountVectorizer` hace posible esta representación vectorial:

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

El método `fit` del vectorizador básicamente recopila el vocabulario, recibiendo como entrada una lista de strings: 

In [14]:
vect.fit(bards_words)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [15]:
print("Tamaño del vocabulario: {}".format(len(vect.vocabulary_)))
print("Vocabulario:\n {}".format(vect.vocabulary_))

Tamaño del vocabulario: 13
Vocabulario:
 {'the': 9, 'fool': 3, 'doth': 2, 'think': 10, 'he': 4, 'is': 6, 'wise': 12, 'but': 1, 'man': 8, 'knows': 7, 'himself': 5, 'to': 11, 'be': 0}


El método `transform`del vectorizador transforma los documentos en vectores. Puesto que la mayoría de las componentes son cero (todos los términos del vocabulario que **no** están en el documento), la representación más adecuada es mediante *matrices dispersas* de **Scipy** 

In [16]:
bag_of_words = vect.transform(bards_words)
print("bag_of_words: {}".format(repr(bag_of_words)))

bag_of_words: <2x13 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>


En este ejemplo pequeño, podemos ver los arrays no dispersos que equivalen a esta representación:

In [17]:
print("Representación no dispersa de los documentos del ejemplo:\n{}".format(
    bag_of_words.toarray()))

Representación no dispersa de los documentos del ejemplo:
[[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


#### Bag-of-word para las críticas de cine

Aplicamos una transformación equivalente a nuestro caso de las críticas de películas

In [18]:
vect = CountVectorizer().fit(text_train)

In [19]:
X_train = vect.transform(text_train)
print("X_train:\n{}".format(repr(X_train)))

X_train:
<25000x74849 sparse matrix of type '<class 'numpy.int64'>'
	with 3431196 stored elements in Compressed Sparse Row format>


Exploremos un poco el vocabulario que se ha recopilado:

In [20]:
feature_names = vect.get_feature_names()
print("Número de términos en el vocabulario: {}".format(len(feature_names)))
print("Primeras 20 características (términos):\n{}".format(feature_names[:20]))
print("Términos del 20010 al 20030:\n{}".format(feature_names[20010:20030]))
print("Términos cada 2000 posiciones:\n{}".format(feature_names[::2000]))

Número de términos en el vocabulario: 74849
Primeras 20 características (términos):
['00', '000', '0000000000001', '00001', '00015', '000s', '001', '003830', '006', '007', '0079', '0080', '0083', '0093638', '00am', '00pm', '00s', '01', '01pm', '02']
Términos del 20010 al 20030:
['dratted', 'draub', 'draught', 'draughts', 'draughtswoman', 'draw', 'drawback', 'drawbacks', 'drawer', 'drawers', 'drawing', 'drawings', 'drawl', 'drawled', 'drawling', 'drawn', 'draws', 'draza', 'dre', 'drea']
Términos cada 2000 posiciones:
['00', 'aesir', 'aquarian', 'barking', 'blustering', 'bête', 'chicanery', 'condensing', 'cunning', 'detox', 'draper', 'enshrined', 'favorit', 'freezer', 'goldman', 'hasan', 'huitieme', 'intelligible', 'kantrowitz', 'lawful', 'maars', 'megalunged', 'mostey', 'norrland', 'padilla', 'pincher', 'promisingly', 'receptionist', 'rivals', 'schnaas', 'shunning', 'sparse', 'subset', 'temptations', 'treatises', 'unproven', 'walkman', 'xylophonist']


### Clasificacion textos usando Naive Bayes Multinomial

Una vez que ya hemos visto cómo vectorizar los documentos, ya podemos aplicar el clasificador `MultinomialNB` para obtener predicciones sobre el "sentimiento" de una crítica:

In [21]:
from sklearn.naive_bayes import MultinomialNB
# Regresión logística con el parámetro por defecto
multinb=MultinomialNB().fit(X_train,y_train)

Basicamente, lo que se ha hecho al entrenar el modelo, es contar términos por cada clase, y calcular el logaritmo de las proporciones. Esto lo podemos consultar

In [22]:
print(multinb.class_count_)
print(multinb.class_log_prior_)
print(multinb.feature_count_)
print(multinb.feature_log_prob_)

[12500. 12500.]
[-0.69314718 -0.69314718]
[[ 51. 174.   1. ...   0.   3.   1.]
 [ 42. 126.   0. ...   1.   1.   0.]]
[[-10.90247427  -9.68893202 -14.16057081 ... -14.85371799 -13.46742363
  -14.16057081]
 [-11.11979709 -10.03681011 -14.8809972  ... -14.18785002 -14.18785002
  -14.8809972 ]]


Veamos cómo se comporta el modelo entrenado en la clasificación de un par de documentos del test. En concreto en la crítica segunda y tercera. La segunda del test es una crítica negativa, y la tercera es positiva:

In [23]:
print("Segunda crítica del conjunto de test: \n\n{}\n".format(text_test[1]))
print("Clasificación verdadera: {}.\n\n".format(y_test[1]))

print("Tercera crítica del conjunto de test: \n\n{}\n".format(text_test[2]))
print("Clasificación verdadera: {}".format(y_test[2]))

Segunda crítica del conjunto de test: 

b'I don\'t know how this movie has received so many positive comments. One can call it "artistic" and "beautifully filmed", but those things don\'t make up for the empty plot that was filled with sexual innuendos. I wish I had not wasted my time to watch this movie. Rather than being biographical, it was a poor excuse for promoting strange and lewd behavior. It was just another Hollywood attempt to convince us that that kind of life is normal and OK. From the very beginning I asked my self what was the point of this movie,and I continued watching, hoping that it would change and was quite disappointed that it continued in the same vein. I am so glad I did not spend the money to see this in a theater!'

Clasificación verdadera: 0.


Tercera crítica del conjunto de test: 

b"I caught this movie on the Horror Channel and was quite impressed by the film's Gothic atmosphere and tone. As a big fan of all things vampire related, I am always happy to see

Veamos cómo los clasifica nuestro clasificador `multinb`. Para ello, primero tenemos que transformarlos a vectores, y luego aplicar la predicción con clasificador:

In [24]:
print("Predicción del clasificador para la segunda crítica: {}\n".format(multinb.predict(vect.transform([text_test[1]]))[0]))

print("Predicción del clasificador para la tercera crítica: {}".format(multinb.predict(vect.transform([text_test[2]]))[0]))

Predicción del clasificador para la segunda crítica: 0

Predicción del clasificador para la tercera crítica: 1


¡Perfecto!, el clasificador ha sido capaz de acertar qué tipo de críticas eran. Con `predict_proba`, podemos saber cómo de seguro estaba de su predicción. Como se ve, estaba bastante seguro en ambos casos:

In [25]:
print("Predicción de probabilidad para la segunda crítica: {}\n".format(multinb.predict_proba(vect.transform([text_test[1]]))[0]))

print("Predicción de probabilidad para la tercera crítica: {}".format(multinb.predict_proba(vect.transform([text_test[2]]))[0]))

Predicción de probabilidad para la segunda crítica: [9.99999862e-01 1.37575474e-07]

Predicción de probabilidad para la tercera crítica: [0.01977596 0.98022404]


Sin embargo, no necesariamente acierta en todas las revisiones. Por ejemplo, la primera crítica es positiva y el clasificador la clasifica como negativa:

In [26]:
print("Primera crítica del conjunto de test: \n\n{}\n".format(text_test[0]))
print("Clasificación verdadera: {}.\n".format(y_test[0]))

print("Predicción del clasificador para la primera crítica: {}\n".format(multinb.predict(vect.transform([text_test[0]]))[0]))

print("Predicción de probabilidad para la primera crítica: {}".format(multinb.predict_proba(vect.transform([text_test[0]]))[0]))

Primera crítica del conjunto de test: 

b"Don't hate Heather Graham because she's beautiful, hate her because she's fun to watch in this movie. Like the hip clothing and funky surroundings, the actors in this flick work well together. Casey Affleck is hysterical and Heather Graham literally lights up the screen. The minor characters - Goran Visnjic {sigh} and Patricia Velazquez are as TALENTED as they are gorgeous. Congratulations Miramax & Director Lisa Krueger!"

Clasificación verdadera: 1.

Predicción del clasificador para la primera crítica: 0

Predicción de probabilidad para la primera crítica: [0.68716538 0.31283462]


-------------------------
Veamos el **rendimiento global** del clasificador **sobre los conjuntos de entrenamiento y test**, que no está nada mal:

In [27]:
X_test = vect.transform(text_test)
print("Rendimiento de multinb sobre el conjunto de entrenamiento: {:.2f}".format(multinb.score(X_train,y_train)))
print("Rendimiento de multinb sobre el conjunto de test: {:.2f}".format(multinb.score(X_test,y_test)))

Rendimiento de multinb sobre el conjunto de entrenamiento: 0.90
Rendimiento de multinb sobre el conjunto de test: 0.81


Por defecto, el parámeto de suavizado `alpha` es 1.0. Podemos usar probar con distintos valores de suavizado (cambiando la cantidad de regularización), a ver qué pasa. Por ejemplo:

In [28]:
multinb_alpha=MultinomialNB(alpha=10).fit(X_train,y_train)
print("Rendimiento de multinb sobre el conjunto de entrenamiento {:.2f}".format(multinb_alpha.score(X_train,y_train)))
print("Rendimiento de multinb sobre el conjunto de test: {:.2f}".format(multinb_alpha.score(X_test,y_test)))

Rendimiento de multinb sobre el conjunto de entrenamiento 0.87
Rendimiento de multinb sobre el conjunto de test: 0.82


Es interesante ver qué ocurre cuando se van cambiando los distintos valores de `alpha` (¿por qué?) (¿cómo influye `alpha` en la regularización?). Para ello podemos usar validación cruzada sobre el conjunto de entrenamiento, y decidir con eso cuál es el mejor `alpha` de un conjunto de candidatos. La utilidad `GridSearchCV` nos automatiza todo el proceso:

In [29]:
from sklearn.model_selection import GridSearchCV
param_grid_nb = {'alpha': [0.0001,0.001, 0.01,0.1, 1, 10,100,200]}
grid_nb = GridSearchCV(MultinomialNB(), param_grid_nb, cv=5)
grid_nb.fit(X_train, y_train)
print("Mejor parámetro: ", grid_nb.best_params_)
print("Rendimiento de MultonomialNB en validación cruzada, con el mejor parámetro: {:.2f}".format(grid_nb.best_score_))

Mejor parámetro:  {'alpha': 0.1}
Rendimiento de MultonomialNB en validación cruzada, con el mejor parámetro: 0.85


De entre los `alpha` candidatos, se ha encontrado que el mejor es el que corresponde al valor de suavizado 0.1. Hay que recordar que `GridSearchCV` es en sí mismo un clasificador, que al entrenarse (`fit`) queda con el clasificador que resulta de usar el mejor de todos los parámetros. Y también tenemos métodos `predict`, `predict`, `predict_proba` y `score`. 

Para dar una evaluación final, miramos la tasa de aciertos sobre el conjunto de prueba. Como se observa no necesariamente es mejor que el mejor rendimiento de los parámetos del grid sobre el conjunto de prueba:

In [30]:
print("Rendimiento sobre prueba (del mejor parámetro en validación cruzada): {:.2f}".format(grid_nb.score(X_test, y_test)))

Rendimiento sobre prueba (del mejor parámetro en validación cruzada): 0.80


## Mejorando la vectorización: *stop words*, `min_df`

Los *stop words* son palabras de uso tan frecuente que no aportan nada a la clasificación de textos (ya que no dan información sobre la clase a la que se pertenece). Igualmente, aquellos términos de muy baja frecuencia podrían ignorarse y así ganar en eficiencia (se tendrían menos características). Las opciones `min_df` y `stop_words` del vectorizador nos permiten llevar a cabo esto: 

In [31]:
vect2 = CountVectorizer(min_df=100, stop_words="english").fit(text_train)
X2_train = vect2.transform(text_train)

Obsérvese como se reduce drásticamente el número de características:

In [32]:
print("Número de términos en el vocabulario original: {}".format(len(feature_names)))
feature_names2 = vect2.get_feature_names()
print("Número de términos en el vocabulario con stop words y min_df: {}".format(len(feature_names2)))

Número de términos en el vocabulario original: 74849
Número de términos en el vocabulario con stop words y min_df: 3561


Obtengamos un clasificador para esta nueva vectorización, y vemos cómo sube (algo) el rendimiento:

In [33]:
multinb2=MultinomialNB(alpha=0.1).fit(X2_train,y_train)

In [34]:
X2_test = vect2.transform(text_test)

print("Rendimiento de multinb2 sobre el conjunto de entrenamiento {:.2f}".format(multinb2.score(X2_train,y_train)))
print("Rendimiento de multinb2 sobre el conjunto de test: {:.2f}".format(multinb2.score(X2_test,y_test)))

Rendimiento de multinb2 sobre el conjunto de entrenamiento 0.85
Rendimiento de multinb2 sobre el conjunto de test: 0.84


Hagamos ahora un tratamiento ahora con `GridSearchCV`, similar a lo que hemos hecho, pero con esta nueva vectorización. Hay que señalar que podría ser posible buscar también valores buenos para los parámetros `min_df` y alguno más de la vectorización, pero para eso deberíamos haber visto antes los *pipelines* de scikit learn. Nos conformamos de momentos con buscar un valor para `alpha`:


In [35]:
grid2_nb = GridSearchCV(MultinomialNB(), param_grid_nb, cv=5)
grid2_nb.fit(X2_train, y_train)
print("Mejor parámetro: ", grid2_nb.best_params_)
print("Rendimiento de MultonomialNB (con min_df y stop words) en validación cruzada, con el mejor parámetro: {:.2f}".format(grid2_nb.best_score_))
print("Rendimiento sobre prueba (del mejor parámetro en validación cruzada): {:.2f}".format(grid2_nb.score(X2_test, y_test)))

Mejor parámetro:  {'alpha': 0.0001}
Rendimiento de MultonomialNB (con min_df y stop words) en validación cruzada, con el mejor parámetro: 0.84
Rendimiento sobre prueba (del mejor parámetro en validación cruzada): 0.84
